# setup

In [ ]:
use role tpch_developer;

# 4.1 Create Sensitive Data

In [ ]:
USE ROLE TPCH_ADMIN;

CREATE OR REPLACE TABLE ANALYTICS.CUSTOMER_SENSITIVE AS
SELECT 
    C_CUSTKEY,
    C_NAME,
    C_ADDRESS,
    C_PHONE,
    C_ACCTBAL,
    C_NATION_NAME,
    C_REGION_NAME,
    C_MKTSEGMENT,
    -- Add sensitive info (simulation)
    'customer_' || C_CUSTKEY || '@company.com' AS EMAIL,
    LPAD(ABS(MOD(C_CUSTKEY * 123456789, 1000000000)), 9, '0') AS SSN_LAST_9, -- Social Security Number
    UNIFORM(100000000::INT, 999999999::INT, RANDOM()) AS PHONE_NUMBER
    
FROM TPCH_ANALYTICS_DB.ANALYTICS.CUSTOMER_SILVER;

# 4.2 Create Masking Policies

In [ ]:
USE ROLE TPCH_ADMIN;

-- ============================================================================
-- STEP 6: Create Masking Policies for Sensitive Columns
-- ============================================================================

-- Masking Policy for Email (show full to ADMIN, partial to ANALYST, mask for SUPPORT)
CREATE OR REPLACE MASKING POLICY CONTROL.MASK_EMAIL AS (val STRING) RETURNS STRING ->
    CASE 
        WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN', 'TPCH_ADMIN', 'TPCH_DEVELOPER') THEN val
        WHEN CURRENT_ROLE() IN ('TPCH_ANALYST') THEN 
            REGEXP_REPLACE(val, '(^[^@]{2})[^@]+', '\\1***')  -- Show first 2 chars before @
        ELSE '***@masked.com'
    END;

-- Masking Policy for Phone (show full to ADMIN, last 4 digits to ANALYST, mask for SUPPORT)
CREATE OR REPLACE MASKING POLICY CONTROL.MASK_PHONE AS (val STRING) RETURNS STRING ->
    CASE 
        WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN', 'TPCH_ADMIN', 'TPCH_DEVELOPER') THEN val
        WHEN CURRENT_ROLE() IN ('TPCH_ANALYST') THEN 
            'XXX-XXX-' || RIGHT(val, 4)
        ELSE 'XXX-XXX-XXX'
    END;

-- Masking Policy for SSN (full mask except for ADMIN)
CREATE OR REPLACE MASKING POLICY CONTROL.MASK_SSN AS (val STRING) RETURNS STRING ->
    CASE 
        WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN', 'TPCH_ADMIN', 'TPCH_DEVELOPER') THEN val
        ELSE '***-**-****'
    END;


# 4.3 Apply AND Test Masking Policies

In [ ]:
USE ROLE TPCH_ADMIN;

-- ============================================================================
-- STEP 7: Apply Masking Policies to Table Columns
-- ============================================================================

ALTER TABLE ANALYTICS.CUSTOMERS_SENSITIVE 
    MODIFY COLUMN EMAIL SET MASKING POLICY MASK_EMAIL;

ALTER TABLE ANALYTICS.CUSTOMERS_SENSITIVE 
    MODIFY COLUMN PHONE_NUMBER SET MASKING POLICY MASK_PHONE;

ALTER TABLE ANALYTICS.CUSTOMERS_SENSITIVE 
    MODIFY COLUMN SSN_LAST_9 SET MASKING POLICY MASK_SSN;


-- ============================================================================
-- STEP 8: Test Masking Policies with Different Roles
-- ============================================================================

-- View all masking policies
SHOW MASKING POLICIES;

-- Describe table to see which columns have masking policies
DESCRIBE TABLE ANALYTICS.CUSTOMERS_SENSITIVE;

-- Test as ROLE_ADMIN (sees everything)
USE ROLE TPCH_DEVELOPER;
SELECT * FROM CUSTOMERS_SENSITIVE;

-- Test as ROLE_ANALYST (sees partial data)
USE ROLE TPCH_ANALYST;
SELECT * FROM CUSTOMERS_SENSITIVE ;

-- Test as ROLE_SUPPORT (most data masked)
USE ROLE TPCH_VIEWER;
SELECT * FROM CUSTOMERS_SENSITIVE;


# 4.4 Secure Data Sharing

In [ ]:
-- ============================================================================
-- SECTION 4: SECURE DATA SHARING
-- ============================================================================
-- Share curated data with external partners/consumers

-- ============================================================================
-- STEP 9: Prepare Data for Sharing
-- ============================================================================

USE ROLE TPCH_DEVELOPER;

-- Create a clean, curated view for sharing (without sensitive data)
CREATE OR REPLACE SECURE VIEW REPORTS.VW_CUSTOMER_PUBLIC AS
SELECT 
    C_CUSTKEY,
    C_NAME,
    C_NATION_NAME,
    C_REGION_NAME,
    C_MKTSEGMENT
FROM ANALYTICS.CUSTOMER_SENSITIVE;
-- Note: Sensitive fields (EMAIL, PHONE, SSN) are excluded

-- Create aggregated view (safe for external sharing)
CREATE OR REPLACE SECURE VIEW REPORTS.VW_CUSTOMER_SUMMARY AS
SELECT 
    C_NATION_NAME,
    C_REGION_NAME,
    C_MKTSEGMENT,
    COUNT(*) AS TOTAL_CUSTOMERS

FROM ANALYTICS.CUSTOMER_SENSITIVE
GROUP BY 1, 2, 3;

-- Verify views
SELECT * FROM REPORTS.VW_CUSTOMER_PUBLIC LIMIT 10;
SELECT * FROM REPORTS.VW_CUSTOMER_SUMMARY LIMIT 10;




In [ ]:

-- ============================================================================
-- STEP 10: Create Secure Share
-- ============================================================================

USE ROLE ACCOUNTADMIN;

-- Create a share (container for sharing objects)
CREATE OR REPLACE SHARE SHARE_CUSTOMER_DATA
    COMMENT = 'Curated customer data for external partners';




In [ ]:
-- Grant usage on database to share
GRANT USAGE ON DATABASE TPCH_ANALYTICS_DB TO SHARE SHARE_CUSTOMER_DATA;

-- Grant usage on schema to share
GRANT USAGE ON SCHEMA TPCH_ANALYTICS_DB.REPORTS TO SHARE SHARE_CUSTOMER_DATA;

-- Grant SELECT on views to share (NOT raw tables!)
GRANT SELECT ON VIEW TPCH_ANALYTICS_DB.REPORTS.VW_CUSTOMER_PUBLIC 
    TO SHARE SHARE_CUSTOMER_DATA;

GRANT SELECT ON VIEW TPCH_ANALYTICS_DB.REPORTS.VW_CUSTOMER_SUMMARY 
    TO SHARE SHARE_CUSTOMER_DATA;


In [ ]:
-- ============================================================================
-- STEP 11: Manage and Monitor Share
-- ============================================================================

-- View all shares
SHOW SHARES;

In [ ]:
-- Describe share contents
DESC SHARE SHARE_CUSTOMER_DATA;


In [ ]:
ALTER SHARE sales_s ADD ACCOUNTS=<orgname.accountname1>,<orgname.accountname2>;

+----------------------------------+
| status                           |
|----------------------------------|
| Statement executed successfully. |
+----------------------------------+

In [ ]:



-- Add consumer account (in production, replace with actual consumer account)
ALTER SHARE SHARE_CUSTOMER_DATA ADD ACCOUNTS = 'ACC_A', 'ACC_B', 'ACC_C';

-- Remove consumer account (if needed)
ALTER SHARE SHARE_CUSTOMER_DATA REMOVE ACCOUNTS = 'ACC_A', 'ACC_B';

# end